In [ ]:
import os
import re
from collections import defaultdict

DATASET_PATH = '/kaggle/input/imagesoasis/Data'
label_map = {
    'Non Demented': 0,
    'Very mild Dementia': 1,
    'Mild Dementia': 2,
    'Moderate Dementia': 3
}

# Group image paths per patient
patient_dict = defaultdict(list)

for label_name in label_map:
    folder = os.path.join(DATASET_PATH, label_name)
    for fname in os.listdir(folder):
        match = re.match(r"(OAS1_\d+)_MR1", fname)
        if match:
            patient_id = match.group(1)
            full_path = os.path.join(folder, fname)
            patient_dict[(patient_id, label_map[label_name])].append(full_path)

# Sort slice paths per patient (important for volume stacking)
for key in patient_dict:
    patient_dict[key] = sorted(patient_dict[key])

print(f"Total patients found: {len(patient_dict)}")


In [ ]:
import numpy as np
import cv2
from tqdm import tqdm

IMG_SIZE = 128
DEPTH = 60  # Number of slices per volume

X_volumes = []
y_labels = []

for (pid, label), slices in tqdm(patient_dict.items()):
    volume = []

    for path in slices:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        volume.append(img)

    volume = np.array(volume)

    # Pad or crop to DEPTH
    if volume.shape[0] < DEPTH:
        pad_width = DEPTH - volume.shape[0]
        pad = np.zeros((pad_width, IMG_SIZE, IMG_SIZE))
        volume = np.concatenate([volume, pad], axis=0)
    else:
        mid = volume.shape[0] // 2
        start = max(0, mid - DEPTH // 2)
        volume = volume[start:start+DEPTH]

    # Normalize and store
    volume = volume.astype('float32') / 255.0
    X_volumes.append(volume)
    y_labels.append(label)

X_volumes = np.stack(X_volumes)
y_labels = np.array(y_labels)
print(f"X shape: {X_volumes.shape}, y shape: {y_labels.shape}")


In [ ]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_volumes, y_labels, test_size=0.2, stratify=y_labels, random_state=42
)

# Convert to PyTorch tensors and reshape for 3D CNN: (N, 1, D, H, W)
X_train_tensor = torch.tensor(X_train).unsqueeze(1).float()
X_test_tensor = torch.tensor(X_test).unsqueeze(1).float()
y_train_tensor = torch.tensor(y_train).long()
y_test_tensor = torch.tensor(y_test).long()

# DataLoader
train_ds = TensorDataset(X_train_tensor, y_train_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=4, shuffle=False)

print(f"Train loader size: {len(train_loader)}, Test loader size: {len(test_loader)}")


In [ ]:
import torch.nn as nn
from torchvision.models.video import r3d_18

class DenseNet3D(nn.Module):
    def __init__(self, num_classes=4):
        super(DenseNet3D, self).__init__()
        # Use 3D ResNet-18 (you can replace this with DenseNet3D later)
        self.backbone = r3d_18(pretrained=False)
        self.backbone.stem[0] = nn.Conv3d(1, 64, kernel_size=(3,7,7), stride=(1,2,2), padding=(1,3,3), bias=False)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)

# Instantiate model
model = DenseNet3D().to("cuda")


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss, correct, total = 0, 0, 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to("cuda"), y_batch.to("cuda")

        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Train Acc: {acc:.2f}%")


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set model to evaluation mode
model.eval()
y_true, y_pred = [], []

# Run predictions on test set
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to("cuda")
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()

        y_pred.extend(preds)
        y_true.extend(y_batch.numpy())

# Accuracy
test_acc = accuracy_score(y_true, y_pred)
print(f"\nTest Accuracy: {test_acc:.4f}\n")

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[k for i, k in enumerate(label_map.keys()) if i in np.unique(y_true)],
            yticklabels=[k for i, k in enumerate(label_map.keys()) if i in np.unique(y_true)])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# Classification report (auto-select valid labels only)
unique_labels = np.unique(y_true + y_pred)
inv_label_map = {v: k for k, v in label_map.items()}
target_names = [inv_label_map[i] for i in unique_labels]

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=target_names))


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

# One-hot encode true labels
n_classes = len(np.unique(y_true))
y_true_bin = label_binarize(y_true, classes=sorted(list(np.unique(y_true))))

# Get softmax scores
y_scores = []

model.eval()
with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to("cuda")
        probs = torch.softmax(model(X_batch), dim=1)
        y_scores.extend(probs.cpu().numpy())

y_scores = np.array(y_scores)

# Compute ROC curve and AUC
fpr, tpr, roc_auc = {}, {}, {}
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(8, 6))
colors = cycle(['blue', 'red', 'green', 'orange'])
inv_label_map = {v: k for k, v in label_map.items()}
labels_present = sorted(np.unique(y_true))

for i, color in zip(range(n_classes), colors):
    label_name = inv_label_map[labels_present[i]]
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'{label_name} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-Class ROC Curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()


In [ ]:
torch.save(model.state_dict(), "multimask_densenet3d_alzheimer.pth")